In [1]:
using MacroTools

https://www.cs.cornell.edu/~ross/publications/eqsat/

Tate thesis chpoater 7 and 8.

We need to tie the knot.
It's a little strange, why is this fixedpoint ok? It must be coinductive.

a = gensym()
t = term!( yadaydayday(a))
assert(a == t)
I think this might do it.
This is a curious analog of landin's knot.



let fact0 = ref (fun x -> x) in
let fact x = if x = 0 then 1 else x * !fact0 (x - 1) in
let () = fact0 := fact


if we weant to work at a lower level IR


The "phi" function is just an if-the-else expression
Kind of the goal is to convert an imperative program into a functional one
loops becomes operations on streams
if statements can be converted to if  expresions

So why even write the imperative program in the first place? Because you are incorrigible

And then the backwards trasnformation is intertestingf. Converting functional code into imperative.

The julia compiler may possibly do all thse trasnfromations anyhow


aexpr and bexpr.
ite(b,x,x) -> x  is a good one
All arithmetic x + y = y + xx etc
all logic 

allessandro already has these. He also made an interpeter? so maybe 


the capture macro is neat. I suppose it generates it's patterns in kind of an obvious way. It probably just ignores lines nums, comments, and :block

dominators in CFG seem important for loop reconstruction and if then else reconstruction..


In [104]:
quote if c_; @invars_#=funbly =# end end

quote
    #= In[104]:1 =#
    if c_
        #= In[104]:1 =#
        s_
    end
end

In [2]:
dump(quote
    if x > 0
        x = 3
    else
            x = 4
    end
    end)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[2]
    2: Expr
      head: Symbol if
      args: Array{Any}((3,))
        1: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol >
            2: Symbol x
            3: Int64 0
        2: Expr
          head: Symbol block
          args: Array{Any}((2,))
            1: LineNumberNode
              line: Int64 3
              file: Symbol In[2]
            2: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Symbol x
                2: Int64 3
        3: Expr
          head: Symbol block
          args: Array{Any}((2,))
            1: LineNumberNode
              line: Int64 5
              file: Symbol In[2]
            2: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Symbol x
                2: Int64 4


In [6]:
dump(ex)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[5]
    2: Expr
      head: Symbol if
      args: Array{Any}((3,))
        1: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol >
            2: Symbol x
            3: Int64 0
        2: Expr
          head: Symbol block
          args: Array{Any}((2,))
            1: LineNumberNode
              line: Int64 3
              file: Symbol In[5]
            2: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Symbol x
                2: Int64 3
        3: Expr
          head: Symbol block
          args: Array{Any}((2,))
            1: LineNumberNode
              line: Int64 5
              file: Symbol In[5]
            2: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Symbol x
                2: Int64 4


In [31]:
ex = quote
    if x > 0
        x = 3
    else
        x = 4
    end
end

ex = quote
     xx = 4
     x = 3
     y = 4
end


    

#s1
#s2

true

In [87]:
dump(quote
    function foo(x,y,z)
        return 7
    end
    end)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[87]
    2: Expr
      head: Symbol function
      args: Array{Any}((2,))
        1: Expr
          head: Symbol call
          args: Array{Any}((4,))
            1: Symbol foo
            2: Symbol x
            3: Symbol y
            4: Symbol z
        2: Expr
          head: Symbol block
          args: Array{Any}((3,))
            1: LineNumberNode
              line: Int64 2
              file: Symbol In[87]
            2: LineNumberNode
              line: Int64 3
              file: Symbol In[87]
            3: Expr
              head: Symbol return
              args: Array{Any}((1,))
                1: Int64 7


In [99]:
function translate_prog(p)
    @capture(p, function f_(args__) body_ end)
    translate_statement(body, Dict(zip(args,args)), 0)[:retvar]
end


function translate_statement(s, env, loop_level) # translate statement
    #println(s)
    if @capture(s, begin s1_; s2__ end) && length(s2) > 0
        return translate_statement( Expr(:block, s2...), translate_statement(s1, env, loop_level), loop_level)
    elseif @capture(s, x_ = e_)
        env2 = copy(env)
        env2[x] = translate_expr(e, env)
        return env2
    elseif @capture(s, if e_ s1_ else s2_ end)
        return phi( translate_expr(e, env), translate_statement(s1, env, loop_level) , translate_statement(s2, env, loop_level) )
    elseif @capture(s, while c_ b_ end)
        
    end
    println("no match", e)
end

function translate_expr(e, env)
    if @capture( e , op_(args__))
        args = [translate_expr(arg, env) for arg in args]
        :($op( $(args...) ))
    elseif e isa Number
        e
    else
        env[e]
    end
end

function combine(m1,m2,conflict)
    res = Dict()
    for (k,v) in m1
        res[k] = v
    end
    for (k,v) in m2
        if haskey(res,k)
            res[k] = conflict( m1[k], m2[k] )
        else
           res[k] = v     
        end
    end
    return res
end

function phi(n, env1, env2,)
    combine(env1,env2, (t,f) -> :(ϕ($n, $t, $f )  ))
end
#function theta()



phi (generic function with 1 method)

In [101]:
ex = quote
    function foo(x,y)
    #z = 4
        if x > 0
            x = 3
            y = 8
        else
            x = 8
            y = 3
        end
        retvar = x + y
    end
end
# This example it would be nice to discover we can remove the branch and phi nodes.
#translate_statement(ex, Dict(), 0)
translate_prog(ex)

:(ϕ(x > 0, 3, 8) + ϕ(x > 0, 8, 3))